In [8]:
# Dependencies

import pandas as pd
import geopandas as gpd

from geopandas import GeoDataFrame
from shapely.geometry import Point
import fiona

In [9]:


# Load file and set first row as index
df = pd.read_csv('data/standard_first_20.csv', encoding='cp1252')

# ##Set first row as column headers
# df.columns = df.iloc[0]

# ## Drop first row and save 
# df = df.drop(index=0)

# # Drop rows with NaN in the 'FOP' Column

# df = df.dropna(subset=['FOP'])
# df

clean_df = df
# clean_df.head()



# Change all columns to string

clean_df = clean_df.astype('string')

clean_df.dtypes

School    string
FOP       string
Foul      string
dtype: object

In [10]:
# Output cleaned df as csv
clean_df.to_csv('data/clean.csv')

df = pd.read_csv('data/clean.csv')
df.dtypes

Unnamed: 0     int64
School        object
FOP           object
Foul          object
dtype: object

In [11]:



### Use Shapely to work with wrt values (the format of the current string POLYGONS)

#You need to convert your pandas DataFrame to a geopandas GeoDataFrame to access the geometry column as shapely object. This post describes how to achieve it.

#one solution:
#https://gis.stackexchange.com/questions/404299/how-to-convert-string-type-to-shapely-object

from shapely import wkt
df = pd.read_csv('data/clean.csv')
fair_df = pd.DataFrame(df, columns=['School','FOP'])
foul_df = pd.DataFrame(df, columns=['School','Foul'])
# foul_df

### Loading the strings as geo objects
fair_df['FOP'] = fair_df['FOP'].apply(wkt.loads)
foul_df['Foul'] = foul_df['Foul'].apply(wkt.loads)

## Put existing dataframes into geodataframes


fair_gdf = gpd.GeoDataFrame(fair_df, columns=['School','FOP'], geometry=fair_df['FOP'], crs='epsg:4326')
foul_gdf = gpd.GeoDataFrame(foul_df, columns=['School','Foul'], geometry=foul_df['Foul'], crs='epsg:4326')

fair_gdf.head()
foul_gdf.head()

# # Create a seperate geodataframe and set the geometry to the Foul column
# gdf_foul = gpd.GeoDataFrame(df, columns=['School','Foul','Foul2'], geometry=df['Foul'], crs='epsg:4326')



#### DOESNT WORK WHEN TRYING TO LOAD THE FINAL COLUMN< NOT SURE WHY
#### THERE ARE A NUMBER OF EMPTY FIELDS 
#df['Foul2'] = df['Foul2'].apply(wkt.loads)





,School,Foul,geometry
0,Charlotte High School,"POLYGON ((-84.8551165 42.5604691, -84.8539296 ...","POLYGON ((-84.85512 42.56047, -84.85393 42.560..."
1,Dewitt HS,"POLYGON ((-84.5810277 42.8262729, -84.5821569 ...","POLYGON ((-84.58103 42.82627, -84.58216 42.826..."
2,Eastern HS,"POLYGON ((-84.5355748 42.7372929, -84.5363943 ...","POLYGON ((-84.53557 42.73729, -84.53639 42.737..."
3,Eaton Rapids 1,"POLYGON ((-84.635145 42.5118192, -84.6360033 4...","POLYGON ((-84.63514 42.51182, -84.63600 42.512..."
4,Eaton Rapids 2,"POLYGON ((-84.6350135 42.5110331, -84.6342571 ...","POLYGON ((-84.63501 42.51103, -84.63426 42.510..."


In [ ]:
gdf_foul.head()

In [12]:
# Calculate Fair area in square feet and export just that value to a new dataframe
# Works
fair_gdf['FOP_area'] = fair_gdf['geometry'].to_crs({'init': 'epsg:4326'})\
               .map(lambda p: p.area)

area_df = pd.DataFrame(fair_gdf, columns=['School','FOP_area'])
area_df.head()
#gdf.head()

c:\Users\Justin\anaconda3\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,School,FOP_area
0,Charlotte High School,8.498999e-07
1,Dewitt HS,8.671026e-07
2,Eastern HS,8.870002e-07
3,Eaton Rapids 1,1.032290e-06
4,Eaton Rapids 2,8.684990e-07


In [13]:
# Calculate Foul area in square feet and export just that value to a new dataframe
#started with epsg:3395

foul_gdf['Foul_area'] = foul_gdf['geometry'].to_crs({'init': 'epsg:4326'})\
              .map(lambda p: p.area)

area_foul_df = pd.DataFrame(foul_gdf, columns =['School','Foul_area'])
area_foul_df.head()


c:\Users\Justin\anaconda3\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,School,Foul_area
0,Charlotte High School,3.631508e-07
1,Dewitt HS,2.089466e-07
2,Eastern HS,3.912354e-07
3,Eaton Rapids 1,3.785751e-07
4,Eaton Rapids 2,3.468754e-07


In [15]:
# Join the two Simple Dataframes

area_df = area_df.merge(area_foul_df, on='School')

area_df.head()
### Tuesday Night -

#############################################################

area_df.to_csv('data/area_output_NEW.csv')


In [ ]:
area_df

# ### Output a csv

#area_df.to_csv('data/area_output.csv')

In [ ]:
########## PLAY AROUND WITH MATPLOT LIB



%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np


#### Bar PLOT
# plot
y = area_df['FOP_area']
x = np.random.uniform(2, 7, len(x))
fig, ax = plt.subplots()

ax.bar(x, y, width=1, edgecolor="white", linewidth=0.7)

ax.set(xlim=(0, 8), xticks=np.arange(1, 8),
       ylim=(0, 2000), yticks=np.arange(1, 8))

plt.show()

# fig = plt.figure()
# ax = plt.axes()


# x = area_df['FOP_area']
# #y = area_df['Foul_area']

# ax.plot(x);

In [ ]:


# Calculate a geodesic area, which is very accurate and only requires an ellipsoid (not a projection). This can be done with pyproj 2.3.0 or later.

# from pyproj import Geod
# from shapely import wkt

# # specify a named ellipsoid
# geod = Geod(ellps="WGS84")

# poly = wkt.loads('''\
# POLYGON ((-116.904 43.371, -116.823 43.389, -116.895 43.407,
# -116.908 43.375, -116.904 43.371))''')

# area = abs(geod.geometry_area_perimeter(poly)[0])

# print('# Geodesic area: {:.3f} m^2'.format(area))

# # # Geodesic area: 13205034.647 m^2

# abs() is used to return only positive areas. A negative area may be returned depending on the winding direction of the polygon.


In [ ]:
######## Works to calculate the geometry column now just have to figure out how to repeat calculation over enitre DF & put into new column

# Calculate a geodesic area, which is very accurate and only requires an ellipsoid (not a projection). This can be done with pyproj 2.3.0 or later.


from pyproj import Geod
from shapely import wkt

# specify a named ellipsoid
geod = Geod(ellps="WGS84")

# poly = wkt.loads(str(gdf.iat[0,5]))

# # ### TRY
# poly = wkt.loads(str(gdf.iloc[0:5,5]))

# area = abs(geod.geometry_area_perimeter(poly)[0])

##### KILLS KERNAL
#gdf['FOP_area'] = abs(geod.geometry_area_perimeter(gdf['geometry']))
                      
#print('# Geodesic area: {:.3f} m^2'.format(area))

gdf.head()

In [ ]:




### Try this a hopefully simple way. Using a library called area

## https://stackoverflow.com/questions/4681737/how-to-calculate-the-area-of-a-polygon-on-the-earths-surface-using-python

from area import area


In [ ]:
# from shapely import wkt

# df['FOP'] = gpd.GeoSeries.from_wkt(df['FOP'])
# df['Foul'] = gpd.GeoSeries.from_wkt(df['Foul'])
# df['Foul2'] = gpd.GeoSeries.from_wkt(df['Foul2'])
# crs_df = df


# geo_frame = gpd.GeoDataFrame(crs_df, geometry='FOP')
# tost = geo_frame.set_crs(crs='epsg:6933')

# print(tost.crs)
# #geo_frame.head

In [ ]:
# Create a column for area calculation of each ploygon



#tost= tost.to_crs({'init': 'epsg:3857'})
print(tost.crs)
tost.head(2)

# test['FOP_area'] = test['FOP'].area/ 10**6
# test.head()

In [ ]:
# tost['FOP_area'] = tost['FOP'].area/10**6
# tost.head()